# Propensity score matching study

Creating some sample calculation for propensity score matching.

In [1]:
from scipy.stats import norm, bernoulli

In [2]:
# generating data
s = 10000
# covariate 1
x1 = bernoulli.rvs(p=.1, size=s)
x2 = bernoulli.rvs(p=.1, size=s)

# treatment is caused by x1 only
p1 = .9
p2 = .1
d = bernoulli.rvs(p=p1, size=s) * x1 + bernoulli.rvs(p=p2, size=s) * (1 - x1)

# some dummy caused by treatment but not causing y
x3 = bernoulli.rvs(p=.8, size=s) * d + bernoulli.rvs(p=.2, size=s) * (1 - d)

# outcome is caused by x1, x2 and treatment
# ATE is 1
y = x1 + x2 + d

# naive ATE estimation based on observed data is biased
(y * d).sum() / d.sum() - (y * (1 - d)).sum() / (1 - d).sum()

np.float64(1.5115780854856056)

In [3]:
# calculate propensity score

import statsmodels.api as sm
import numpy as np


# Design matrix with intercept
X = np.column_stack([x1, x2])
X = sm.add_constant(X, has_constant='add')

# Fit logistic regression
model = sm.Logit(d, X).fit()

print(model.summary())


Optimization terminated successfully.
         Current function value: 0.317422
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.3229
Time:                        22:35:52   Log-Likelihood:                -3174.2
converged:                       True   LL-Null:                       -4688.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2577      0.038    -59.753      0.000      -2.332      -2.184
x1             4.4405      0.

In [4]:
# check predictions
X_new = np.array([
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 1]
])
X_new = sm.add_constant(X_new, has_constant='add')  # <-- force constant

model.predict(X_new)
# -> propensity scores are roughly determined by x1
# Note the true weights are 0.9 and 0.1, so we are close

array([0.89869703, 0.91314487, 0.09468759, 0.11028097])

In [5]:
# calcualte ATE based on strata of same propensity score (given by strata of x1)

# mean of treated units with propensity ~ 0.9
m1 = (y * x1 * d).sum() / (x1 * d).sum()
# mean of untreatet units with propensity ~ 0.9
m2 = (y * x1 * (1 - d)).sum() / (x1 * (1 - d)).sum()

# mean of treated units with propensity ~ 0.2
m3 = (y * (1 - x1) * d).sum() / ((1 - x1) * d).sum()
# mean of untreatet units with propensity ~ 0.2
m4 = (y * (1 - x1) * (1 - d)).sum() / ((1 - x1) * (1 - d)).sum()

m1 - m2, m3 - m4
# -> both are close to the real ATE of 1,
# so the estimate on the full population will also be close to 1

(np.float64(1.0205893020973351), np.float64(1.015164390360786))